## Demostración de seguridad usando Raiz primitiva en Problema del Logaritmo Discreto

El **logaritmo discreto** consiste en encontrar el entero \( x \) que satisface:

$$
\
g^x \equiv y \pmod{p}
\
$$

donde:

- \( p \) es un número primo.
- \( g \) es una raíz primitiva de \( p \)
- \( y \) es un entero tal que $ ( y = g^x \mod p ) $.

Existen diferentes métodos para resolver este problema:
1. **Búsqueda Exhaustiva (Brute Force):**  
   Se prueban todos los valores \( x = 0, 1, 2,... p-1 \). Su complejidad es \( O(p) \) y resulta ineficiente para valores grandes.
2. **Baby-step Giant-step (BSGS):**  
   Este algoritmo reduce la complejidad a \( O($\sqrt{p}$) \) dividiendo el problema en dos fases (baby-steps y giant-steps).

## Caso real
La empresa CBA está usando siempre el mismo número $ \alpha $ para realizar todas sus claves de firma en el algoritmo DSA.
Aunque lleva muchos años usando siempre el mismo número. la última auditoria externa realizada en la empresa le han penalizado por usar Firmas Digitales poco seguras en el entorno actual.

### Objetivo
Como criptoanalista debes mostrar en la reunión de dirección porque es será muy positivo implementar un nuevo flujo de trabajo que haga las claves de firma con un $ \alpha $ que sea raiz primitiva con el módulo elegido.

¿Cual sería esa grafica que mostrarias y tu parte explicativa?


In [ ]:
import math
import time
import random
import matplotlib.pyplot as plt
import sympy  # obtener raíces primitivas
import libnum  # para trabajar con números grandes

In [ ]:
def discrete_log_brute_force(g, y, p):
    for x in range(p):
        if pow(g, x, p) == y:
            return x
    return None

# Baby-step Giant-step (BSGS)
def discrete_log_bsgs(g, y, p):
    m = math.isqrt(p - 1) + 1  # m = ceil(sqrt(p-1))
    # Baby-steps: precomputamos g^j for j = 0, 1, ..., m-1
    baby_steps = {}
    for j in range(m):
        baby_steps[pow(g, j, p)] = j

    # Calcular g^(-m) mod p
    c = pow(g, m, p)
    c_inv = pow(c, -1, p)  # O bien usando sympy.mod_inverse(c, p)

    gamma = y
    for i in range(m):
        if gamma in baby_steps:
            return i * m + baby_steps[gamma]
        gamma = (gamma * c_inv) % p
    return None


In [ ]:
# Lista de tamaños en bits para p
bit_sizes = [12, 14, 16]  # Se pueden agregar tamaños mayores (con precaución)
times_bruteforce = [] # tiempos de ejecución para brute force
times_bsgs = []     # tiempos de ejecución para BSGS

# Para cada tamaño en bits, se genera un primo, se obtiene una raíz primitiva,
# se elige un exponente secreto x, se calcula y y se mide el tiempo para hallar x
for bits in bit_sizes:
    p = libnum.generate_prime(bits) # primo de 'bits' bits
    g = sympy.primitive_root(p) # raíz primitiva
    
    x_true = random.randint(1, p-1) # exponente secreto
    y_val = pow(g, x_true, p) # y = g^x mod p
    
    # Medir tiempo para brute force
    start_time = time.perf_counter()
    x_bf = discrete_log_brute_force(g, y_val, p) 
    end_time = time.perf_counter()
    t_bf = end_time - start_time
    times_bruteforce.append(t_bf)
    
    # Medir tiempo para BSGS
    start_time = time.perf_counter()
    x_bsgs = discrete_log_bsgs(g, y_val, p)
    end_time = time.perf_counter()
    t_bsgs = end_time - start_time
    times_bsgs.append(t_bsgs)
    
    print(f"Bits: {bits}, p: {p}, g: {g}, x_true: {x_true}, x_bf: {x_bf}, x_bsgs: {x_bsgs}")
    print(f"Tiempo Brute Force: {t_bf:.6f} s, Tiempo BSGS: {t_bsgs:.6f} s\n")

In [ ]:
# Graficar la comparativa
plt.figure(figsize=(8,6))
plt.plot(bit_sizes, times_bruteforce, marker='o', linestyle='-', color='red', label='Búsqueda Exhaustiva')
plt.plot(bit_sizes, times_bsgs, marker='o', linestyle='-', color='blue', label='Baby-step Giant-step')
plt.xlabel("Tamaño en bits del primo \(p\)")
plt.ylabel("Tiempo de cálculo (s)")
plt.title("Comparativa: Tiempo de cálculo del Logaritmo Discreto")
plt.grid(True)
plt.legend()
plt.show()